In [1]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import glob

# make function

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    return pose

In [22]:
def save_csv(data, name):
    lv1 = [x.name for x in mp_pose.PoseLandmark]
    lv2 = ['x', 'y', 'z', 'v']
    columns = pd.MultiIndex.from_product([lv1, lv2])
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(f"./직촬편집/volcano/{name}_skeleton.csv", index=False)

In [5]:
def make_csv(results, name):
    data = []
    for result in results:
        pose = extract_keypoints(result)

        data.append(pose)

    data = np.array(data)

    save_csv(data, name)

# make skeleton csv files of videos

In [23]:
paths = glob.glob("./직촬편집/volcano/*.mp4")
paths

['./직촬편집/volcano\\(final)volcano_cs.mp4',
 './직촬편집/volcano\\(final)volcano_dy.mp4',
 './직촬편집/volcano\\(final)volcano_sj.mp4',
 './직촬편집/volcano\\(final)volcano_uk.mp4',
 './직촬편집/volcano\\(final)volcano_yj.mp4']

In [24]:
for path in paths:
    name = path.split('(final)')[1].split('.')[0]
    cap = cv2.VideoCapture(path)

    # video info
    fps = cap.get(cv2.CAP_PROP_FPS)
    delay = int(1000/fps)
    print(name)
    print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print("fps :", fps)
    print("------------------------------------------------")

    data = []

    with mp_pose.Pose(static_image_mode=False,
                        model_complexity=1,
                        smooth_landmarks=True,
                        enable_segmentation=False,
                        smooth_segmentation=True,
                        min_detection_confidence=0.8,
                        min_tracking_confidence=0.8,) as pose:

        while cap.isOpened():

            ret, frame = cap.read()

            if not ret:
                break

            img = cv2.resize(frame, (0,0), fx=1/2, fy=1/2, interpolation=cv2.INTER_AREA)
            img = cv2.flip(img, 0)
            
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False
            results = pose.process(img)
            img.flags.writeable = True
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            try:
                landmarks = results.pose_landmarks.landmark
                #print(landmarks)
            except:
                pass

            data.append(results)

            # mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # cv2.imshow("test", img)

            if cv2.waitKey(delay) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    
    make_csv(data, name)

volcano_cs
height : 1080.0
width : 1920.0
fps : 30.0
------------------------------------------------
volcano_dy
height : 1080.0
width : 1920.0
fps : 30.010202038500776
------------------------------------------------
volcano_sj
height : 1080.0
width : 1920.0
fps : 30.0
------------------------------------------------
volcano_uk
height : 1080.0
width : 1920.0
fps : 30.010204750929333
------------------------------------------------
volcano_yj
height : 1080.0
width : 1920.0
fps : 30.0
------------------------------------------------
